In [1]:
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import sumolib
import traci
from sumolib import checkBinary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from utils import *


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
SUMO_HOME found
Using device: cuda


In [2]:
planned_path = get_planned_path()
path_values = list(planned_path.values())

 Retrying in 1 seconds
***Starting server on port 36873 ***
Loading net-file from './config/osm.net.xml.gz' ... done (83ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (223ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 10476.00
Reason: TraCI requested termination.
Performance: 
 Duration: 4.98s
 TraCI-Duration: 3.23s
 Real time factor: 2103.61
 UPS: 64752.610442
Vehicles: 
 Inserted: 1530
 Running: 0
 Waiting: 0
Statistics (avg of 1530):
 RouteLength: 1611.54
 Speed: 7.66
 Duration: 210.76
 WaitingTime: 12.69
 TimeLoss: 40.04
 DepartDelay: 0.55



In [34]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list) / 10

def missing(x, y, z, p):
    return torch.tensor(np.repeat(np.random.rand(x * y) < p, z).reshape(x, y, z)).float()

def generate_masks(tensors, min_mask_ratio=0.2, max_mask_ratio=0.4, missing_ratio=0.5, complete_traj_ratio=0.8):
    initial_masks = missing(tensors.shape[0], tensors.shape[1], tensors.shape[2], missing_ratio)
    masks = []
    for initial_mask in initial_masks:
        if np.random.rand() < complete_traj_ratio:
            masks.append(torch.zeros_like(initial_mask).tolist())
            continue
        seq_length = initial_mask.shape[0]
        mask_start = np.random.randint(int(seq_length * min_mask_ratio), int(seq_length * max_mask_ratio))
        mask = torch.zeros_like(initial_mask)
        mask[:, :mask_start] = 1
        mask = initial_mask * mask
        mask[0] = 0
        mask[1] = 0
        masks.append(mask.tolist())
    return torch.tensor(masks)

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_path_values = torch.tensor(path_values[:train_size]) / 10
val_path_values = torch.tensor(path_values[train_size:]) / 10

train_mask = generate_masks(train_trajectories_tensor)

class DatasetWithPlans(Dataset):
    def __init__(self, tensor, input_mask, path_values):
        self.tensor = tensor.float().to(device)
        self.input_mask = input_mask.float().to(device)
        self.path_values = path_values.float().to(device)
    def __len__(self):
        return len(self.tensor)
    def __getitem__(self, idx):
        return self.tensor[idx], self.input_mask[idx], self.path_values[idx]

train_dataset = DatasetWithPlans(train_trajectories_tensor, train_mask, train_path_values)

In [4]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
        self.fc2 = nn.Linear(int(hidden_size/2), num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.gru(x, h0)  
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out, hidden

class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        mse = torch.mean(((output - target) ** 2) * mask)
        directional_diff = torch.mean(torch.abs(torch.atan2(output[:, 1], output[:, 0]) - torch.atan2(target[:, 1], target[:, 0])))
        return mse + directional_diff

In [48]:
def project_to_nearest(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        return projections

In [49]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks, planned_path in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :]
                if epoch > 30:
                    if epoch > 40:
                        # print(t, epoch)
                        # print("predicted")
                        # print(prediction[0])
                        projected = project_to_nearest(prediction, planned_path)
                        # print("projected")
                        # print(projected[0])
                        # print("actual")
                        # print(inputs[:, t, :][0])
                        if prediction.isnan().any():
                            break
                        current_input = (projected * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                    else:
                        current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 1427475.840625
Epoch 2, Loss: 1383796.70625
Epoch 3, Loss: 1315500.825
Epoch 4, Loss: 1267752.6125
Epoch 5, Loss: 1012911.75
Epoch 6, Loss: 706625.1328125
Epoch 7, Loss: 503822.771875
Epoch 8, Loss: 361986.04375
Epoch 9, Loss: 243861.7859375
Epoch 10, Loss: 165344.46796875
Epoch 11, Loss: 119757.9625
Epoch 12, Loss: 92628.2537109375
Epoch 13, Loss: 77675.51796875
Epoch 14, Loss: 61885.7173828125
Epoch 15, Loss: 43761.770703125
Epoch 16, Loss: 26485.97265625
Epoch 17, Loss: 15335.294970703126
Epoch 18, Loss: 11014.0578125
Epoch 19, Loss: 9820.010693359374
Epoch 20, Loss: 9267.239965820312
Epoch 21, Loss: 8947.022314453125
Epoch 22, Loss: 9106.171923828126
Epoch 23, Loss: 8782.89033203125
Epoch 24, Loss: 8756.668627929688
Epoch 25, Loss: 8686.086547851562
Epoch 26, Loss: 8458.779418945312
Epoch 27, Loss: 8556.994604492187
Epoch 28, Loss: 8537.5310546875
Epoch 29, Loss: 8392.732641601562
Epoch 30, Loss: 8543.315698242188
Epoch 31, Loss: 8410.270703125
Epoch 32, Loss: 11921.

In [56]:
def checkModel(model, inputs, masks, paths):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            projected = project_to_nearest(prediction, paths)
            current_input = (projected * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(projected[2], inputs[:, t, :][2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks, paths = next(iter(val_dataloader))

checkModel(model, inputs, masks, paths)

tensor([90.1600, 24.6880], device='cuda:0') tensor([90.8511, 25.0030], device='cuda:0')
tensor([90.7720, 24.9670], device='cuda:0') tensor([91.1982, 25.1613], device='cuda:0')
tensor([91.4188, 25.2618], device='cuda:0') tensor([91.7239, 25.4009], device='cuda:0')
tensor([92.3640, 25.6927], device='cuda:0') tensor([92.3756, 25.6980], device='cuda:0')
tensor([93.5530, 26.2219], device='cuda:0') tensor([93.1041, 26.0232], device='cuda:0')
tensor([95.2779, 26.4987], device='cuda:0') tensor([93.7963, 26.3286], device='cuda:0')
tensor([96.3856, 27.3196], device='cuda:0') tensor([94.5863, 26.2691], device='cuda:0')
tensor([97.3586, 27.5403], device='cuda:0') tensor([95.2795, 26.4999], device='cuda:0')
tensor([97.7090, 27.3420], device='cuda:0') tensor([95.9272, 26.9799], device='cuda:0')
tensor([97.7720, 27.2587], device='cuda:0') tensor([96.5250, 27.4229], device='cuda:0')
tensor([97.7965, 27.2262], device='cuda:0') tensor([97.1823, 27.6048], device='cuda:0')
tensor([97.8284, 27.1841], devic

In [58]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks, paths in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            projected = project_to_nearest(prediction, paths)
            current_input = (projected * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((projected - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 70.80623387146325
Validation Loss: 35.435041838192376
Validation Loss: 22.416716748333386
Validation Loss: 16.59096758740131
Validation Loss: 13.331092698765016
